In [2]:
import keras
import numpy as np
import os
import random
from keras import backend as K
from keras.layers import Input, Conv2D,GlobalAveragePooling2D, Dense, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model
from keras.layers import DepthwiseConv2D

from keras import optimizers,regularizers
from tensorflow_model_optimization.quantization.keras import vitis_quantize
#from keras.initializers import he_normal



def depthwise_separable(x,params):
    # f1/f2 filter size, s1 stride of conv
    (s1,f2) = params
    x = DepthwiseConv2D((3,3),strides=(s1[0],s1[0]), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(int(f2[0]), (1,1), strides=(1,1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x


def MobileNet(img_input,shallow=False, classes=10):
    """Instantiates the MobileNet.Network has two hyper-parameters
        which are the width of network (controlled by alpha)
        and input size.
        # Arguments
            alpha: optional parameter of the network to change the 
                width of model.
            shallow: optional parameter for making network smaller.
            classes: optional number of classes to classify images
                into.
    """
    x = Conv2D(int(32), (4,4), strides=(2,2), padding='same')(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    filter1_num = random.randrange(32,64,2)
    x = depthwise_separable(x,params=[(1,),(filter1_num,)])
    filter2_num = random.randrange(filter1_num,128,2)
    x = depthwise_separable(x,params=[(2,),(filter2_num,)])
    filter3_num = random.randrange(filter2_num,128,2)
    x = depthwise_separable(x,params=[(1,),(filter3_num,)])
    filter4_num = random.randrange(filter3_num,256,2)
    x = depthwise_separable(x,params=[(2,),(filter4_num,)])
    filter5_num = random.randrange(filter4_num,256,2)
    x = depthwise_separable(x,params=[(1,),(filter5_num,)])
    filter6_num = random.randrange(filter5_num,512,2)
    x = depthwise_separable(x,params=[(2,),(filter6_num,)])
    
    if not shallow:
        for _ in range(5):
            filter7_num = random.randrange(filter6_num,512,2)
            x = depthwise_separable(x,params=[(1,),(filter7_num,)])
    
    filter8_num = random.randrange(filter7_num,1024,2)
    x = depthwise_separable(x,params=[(2,),(filter8_num,)])
    filter9_num = random.randrange(filter8_num,1024,2)
    x = depthwise_separable(x,params=[(2,),(filter9_num,)])

    x = GlobalAveragePooling2D()(x)
    out = Dense(classes, activation='softmax')(x)
    return out

shape_range =range(32,512,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    for step in range(1,11):
        img_input=Input(shape=(shape_x,shape_y,1))
        output = MobileNet(img_input)
        model=Model(img_input,output)
        model.summary()
        model.save('./MobileNet_h5/MobileNet_{input_x}_{input_y}_{step}.h5'.format(input_x=shape_x,input_y=shape_y,step=step))
    
    #model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    #history = model.fit(X_train, Y_train, epochs=1)
    #model.get_weights()[0].dtype
        quantizer = vitis_quantize.VitisQuantizer(model)
        quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:10], weight_bit=8, activation_bit=8)
        #quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
        quantized_model.save('tf2_mnist_classifier_quantized.h5')
        !vai_c_tensorflow2 \
        --model ./tf2_mnist_classifier_quantized.h5 \
        --arch ./arch.json \
        --output_dir ./xmodel_MobileNet \
        --net_name deploy
        os.rename("./xmodel_MobileNet/deploy.xmodel","./xmodel_MobileNet/MobileNet_{shape_x}_{shape_y}_{step}.xmodel".format(shape_x=shape_x,shape_y=shape_y,step=step))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 16, 16, 32)        544       
_________________________________________________________________
batch_normalization_27 (Batc (None, 16, 16, 32)        128       
_________________________________________________________________
activation_27 (Activation)   (None, 16, 16, 32)        0         
_________________________________________________________________
depthwise_conv2d_13 (Depthwi (None, 16, 16, 32)        320       
_________________________________________________________________
batch_normalization_28 (Batc (None, 16, 16, 32)        128       
_________________________________________________________________
activation_28 (Activation)   (None, 16, 16, 32)        0   

[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 4s 397ms/step
[VAI INFO] CrossLayerEqualization Done.


2022-04-30 10:16:40.883960: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 9s 9s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist_classifier_quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 92/92 [00:00<00:00, 3129.62it/s]            
[INFO] dump xmodel: /tmp/deploy_org.xmodel
[UNILOG][INFO] Compile mode: dpu
[UNILOG][INFO] Debug mode: function
[UNILOG][INFO] Target architecture: DPU

[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 4s 390ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 9s 9s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist_classifier_quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 92/92 [00:00<00:00, 3480.91it/s]            
[INFO] dump x

...

[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 5s 513ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 12s 12s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist_classifier_quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 92/92 [00:00<00:00, 3219.98it/s]            
[INFO] dump

[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 5s 516ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 12s 12s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist_classifier_quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 92/92 [00:00<00:00, 3269.39it/s]            
[INFO] dump

[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 5s 508ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 12s 12s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist_classifier_quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 92/92 [00:00<00:00, 3838.15it/s]            
[INFO] dump

[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 5s 515ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 12s 12s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist_classifier_quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 92/92 [00:00<00:00, 4106.85it/s]            
[INFO] dump

[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 5s 514ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 12s 12s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist_classifier_quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 92/92 [00:00<00:00, 3608.27it/s]            
[INFO] dump

In [15]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
import numpy as np
import os
from tensorflow_model_optimization.quantization.keras import vitis_quantize


shape_range =range(32,491,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,3])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,3])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0

# create the base pre-trained model
    base_model = MobileNet(
    input_shape=(shape_x,shape_y,3),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(10, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    model.summary()
    model.save('./MobileNet_h5/MobileNet_{input_x}_{input_y}.h5'.format(input_x=shape_x,input_y=shape_y))
    quantizer = vitis_quantize.VitisQuantizer(model)
    quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
#         quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    quantized_model.save('quantized.h5')
    !vai_c_tensorflow2 \
            --model ./quantized.h5 \
            --arch ./arch.json \
            --output_dir ./xmodel_MobileNet \
            --net_name deploy
    os.rename("./xmodel_MobileNet/deploy.xmodel","./xmodel_MobileNet/MobileNet_{input_x}_{input_y}.xmodel".format(input_x=shape_x,input_y=shape_y))

Model: "model_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_36 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 16, 16, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 16, 16, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 16, 16, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 16, 16, 32)        128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 16, 16, 32)        0  

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 4s 442ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 9s 9s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 4s 450ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 9s 9s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 3s 367ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 9s 9s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 3s 359ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 9s 9s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

...

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 4s 459ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 10s 10s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel    

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 3s 366ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 10s 10s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel    

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 3s 364ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 10s 10s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel    

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 3s 360ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 10s 10s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel    

FileNotFoundError: [Errno 2] No such file or directory: './xmodel_MobileNet/deploy.xmodel' -> './xmodel_MobileNet/MobileNet_288_288.xmodel'